In [10]:
import warnings
warnings.filterwarnings('ignore')
import stwcs
import glob
import sys
import os
import shutil
import time
import filecmp
import astroquery
import progressbar
import copy
import requests
import random
import astropy.wcs as wcs
import numpy as np
from contextlib import contextmanager
from astropy import units as u
from astropy.utils.data import clear_download_cache,download_file
from astropy.io import fits
from astropy.table import Table, Column, unique
from astropy.time import Time
from astroscrappy import detect_cosmics
from stwcs import updatewcs
from scipy.interpolate import interp1d

# Internal dependencies
from common import Constants
from common import Options
from common import Settings
from common import Util
from nbutils import get_filter, get_instrument, get_chip, get_filter
from nbutils import get_zpt, add_visit_info, organize_reduction_tables, pick_deepest_images

@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

with suppress_stdout():
    from drizzlepac import tweakreg,astrodrizzle,catalogs,photeq
    from astroquery.mast import Observations
    from astropy.coordinates import SkyCoord

In [2]:
acceptable_filters = [
    'F220W','F250W','F330W','F344N','F435W','F475W','F550M','F555W',
    'F606W','F625W','F660N','F660N','F775W','F814W','F850LP','F892N',
    'F098M','F105W','F110W','F125W','F126N','F127M','F128N','F130N','F132N',
    'F139M','F140W','F153M','F160W','F164N','F167N','F200LP','F218W','F225W',
    'F275W','F280N','F300X','F336W','F343N','F350LP','F373N','F390M','F390W',
    'F395N','F410M','F438W','F467M','F469N','F475X','F487N','F547M',
    'F600LP','F621M','F625W','F631N','F645N','F656N','F657N','F658N','F665N',
    'F673N','F680N','F689M','F763M','F845M','F953N','F122M','F160BW','F185W',
    'F218W','F255W','F300W','F375N','F380W','F390N','F437N','F439W','F450W',
    'F569W','F588N','F622W','F631N','F673N','F675W','F702W','F785LP','F791W',
    'F953N','F1042M']

### Get obstable

In [3]:
workdir = '.'
input_images = [s for p in ['*flc.fits', '*flt.fits'] for s in glob.glob(os.path.join(workdir, p))]

In [6]:
img = input_images
zptype = 'abmag'
good = []
image_number = []
#check if image exists (check in archive otherwise)
for image in img:
    success = True
    if not os.path.exists(image):
        success = False
    if success:
        good.append(image)
img = copy.copy(good)

hdu = fits.open(img[0])
h = hdu[0].header

exp = [fits.getval(image,'EXPTIME') for image in img] #exposure time
if 'DATE-OBS' in h.keys() and 'TIME-OBS' in h.keys(): 
    dat = [fits.getval(image,'DATE-OBS') + 'T' +
           fits.getval(image,'TIME-OBS') for image in img] #datetime
elif 'EXPSTART' in h.keys():
    dat = [Time(fits.getval(image, 'EXPSTART'),
        format='mjd').datetime.strftime('%Y-%m-%dT%H:%M:%S') #datetime if DATE-OBS is missing
        for image in img]
    
fil = [get_filter(image) for image in img]
ins = [get_instrument(image) for image in img]
det = ['_'.join(get_instrument(image).split('_')[:2]) for image in img]
chip= [get_chip(image) for image in img]
zpt = [get_zpt(i, ccdchip=c, zptype=zptype) for i,c in zip(img,chip)]

if not image_number:
    image_number = [0 for image in img]
    
obstable = Table([img,exp,dat,fil,ins,det,zpt,chip,image_number],
    names=['image','exptime','datetime','filter','instrument',
     'detector','zeropoint','chip','imagenumber'])
obstable.sort('datetime')
obstable = add_visit_info(obstable)

In [7]:
obstable.add_column(Column([' '*99]*len(obstable), name='drizname'))
for i,row in enumerate(obstable):
    visit = row['visit']
    n = str(visit).zfill(4)
    inst = row['instrument']
    filt = row['filter']

    # Visit should correspond to first image so they're all the same
    visittable = obstable[obstable['visit']==visit]
    refimage = visittable['image'][0]
    if 'DATE-OBS' in h.keys():
        date_obj = Time(fits.getval(refimage, 'DATE-OBS'))
    else:
        date_obj = Time(fits.getval(refimage, 'EXPSTART'), format='mjd')
    date_str = date_obj.datetime.strftime('%y%m%d')

    # Make a photpipe-like image name
    drizname = ''
    objname = None
    if objname:
        drizname = '{obj}.{inst}.{filt}.ut{date}_{n}.drz.fits'
        drizname = drizname.format(inst=inst.split('_')[0],
            filt=filt, n=n, date=date_str, obj=objname)
    else:
        drizname = '{inst}.{filt}.ut{date}_{n}.drz.fits'
        drizname = drizname.format(inst=inst.split('_')[0],
            filt=filt, n=n, date=date_str)

    if '.':
        drizname = os.path.join('.', drizname)

    obstable[i]['drizname'] = drizname

In [18]:
obstable = obstable[:12]

In [19]:
obstable

image,exptime,datetime,filter,instrument,detector,zeropoint,chip,imagenumber,visit,drizname
str20,float64,str19,str5,str12,str7,float64,int64,int64,int64,str99
./jbqz22siq_flc.fits,480.0,2012-01-04T08:08:04,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22skq_flc.fits,480.0,2012-01-04T08:18:42,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22smq_flc.fits,480.0,2012-01-04T08:29:20,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22soq_flc.fits,480.0,2012-01-04T08:39:58,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23tqq_flc.fits,480.0,2012-01-04T16:06:48,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23trq_flc.fits,480.0,2012-01-04T16:17:26,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23ttq_flc.fits,480.0,2012-01-04T16:28:04,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23tvq_flc.fits,480.0,2012-01-04T16:38:42,f606w,acs_wfc_full,acs_wfc,26.491377054545822,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz24tyq_flc.fits,480.0,2012-01-04T18:08:11,f606w,acs_wfc_full,acs_wfc,26.491376957830788,1,0,1,./acs.f606w.ut120104_0001.drz.fits
